# Stock NeurIPS2018 Part 3. Backtest - Schritt für Schritt

Wir folgen genau dem NeurIPS2018 Beispiel, aber verwenden Ihre trainierten Modelle und Daten.

## Schritt 1: Notwendige Pakete installieren

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS
from stable_baselines3 import PPO

%matplotlib inline
plt.style.use('default')

## Schritt 2: Daten laden

Wir laden Ihre vorbereiteten Trainingsdaten und Handelsdaten.

In [ ]:
## Schritt 1: Daten laden und vorbereiten
# Daten laden
train = pd.read_csv('train.csv')
trade = pd.read_csv('trade.csv')

# Datum konvertieren
trade['date'] = pd.to_datetime(trade['date'])
trade = trade.sort_values(['date', 'tic']).reset_index(drop=True)

# Technische Indikatoren identifizieren
INDICATORS = ['macd', 'rsi_30', 'cci_30', 'dx_30']  # Passe diese Liste an deine Daten an

# Dimensionen berechnen
stock_dimension = len(trade['tic'].unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension

print(f"Aktien: {trade['tic'].unique()}")
print(f"Aktien Dimension: {stock_dimension}")
print(f"Zustandsraum: {state_space}")
print(f"Verfügbare Spalten: {trade.columns.tolist()}")

## Schritt 2: Daten für numpy-basierte Umgebung vorbereiten
# Pivot-Tabellen für jeden benötigten Wert erstellen
unique_dates = trade['date'].unique()
unique_stocks = trade['tic'].unique()

# Arrays vorbereiten
price_array = np.zeros((len(unique_dates), stock_dimension))
tech_arrays = {ind: np.zeros((len(unique_dates), stock_dimension)) for ind in INDICATORS}

# Daten in Arrays füllen
for i, date in enumerate(unique_dates):
    date_data = trade[trade['date'] == date]
    for j, tic in enumerate(unique_stocks):
        stock_data = date_data[date_data['tic'] == tic]
        if not stock_data.empty:
            price_array[i, j] = stock_data['close'].values[0]
            for ind in INDICATORS:
                if ind in stock_data.columns:
                    tech_arrays[ind][i, j] = stock_data[ind].values[0]

# Tech array kombinieren
tech_array = np.column_stack([tech_arrays[ind] for ind in INDICATORS])

# Turbulenz array
turbulence_array = trade.groupby('date')['turbulence'].first().values

## Schritt 3: Handelsumgebung erstellen
env_config = {
    "price_array": price_array,
    "tech_array": tech_array,
    "turbulence_array": turbulence_array,
    "if_train": False,  # Für Backtesting
    "initial_amount": 1000000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

env_trade = StockTradingEnv(config=env_config)


NameError: name 'pd' is not defined

## Schritt 3: Trainiertes Modell laden

Wir laden das PPO-Modell, das Sie bereits trainiert haben.

In [5]:
# Laden des trainierten PPO Modells
try:
    trained_model = PPO.load("trained_ppo_model")
    print("PPO Modell erfolgreich geladen")
except Exception as e:
    print(f"Fehler beim Laden des Modells: {e}")

PPO Modell erfolgreich geladen


/home/group3/venv/lib/python3.12/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


## Schritt 4: Handelsumgebung vorbereiten

Wir erstellen die Handelsumgebung mit den gleichen Parametern wie im Training.

## Schritt 5: Modell-Vorhersagen durchführen

Jetzt lassen wir das Modell auf den Testdaten handeln.

In [6]:
# Vorhersagen mit dem Modell machen
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_model,
    environment=env_trade
)

print("\nBacktesting Ergebnisse:")
initial_value = df_account_value['account_value'][0]
final_value = df_account_value['account_value'].iloc[-1]
total_return = (final_value - initial_value) / initial_value * 100

print(f"Anfänglicher Portfolio-Wert: ${initial_value:,.2f}")
print(f"Finaler Portfolio-Wert: ${final_value:,.2f}")
print(f"Gesamtrendite: {total_return:.2f}%")

NameError: name 'env_trade' is not defined

## Schritt 6: Dow Jones Industrial Average (DJIA) als Benchmark hinzufügen

In [ ]:
# DJIA Daten herunterladen mit korrektem Datumsformat
start_date = trade.index[0].strftime('%Y-%m-%d')
end_date = trade.index[-1].strftime('%Y-%m-%d')

df_dji = YahooDownloader(
    start_date=start_date,
    end_date=end_date,
    ticker_list=['^DJI']
).fetch_data()

# DJIA Daten verarbeiten
df_dji = df_dji[['date', 'close']]
df_dji['date'] = pd.to_datetime(df_dji['date'])
df_dji = df_dji.set_index('date')
df_dji = df_dji.sort_index()

# Sicherstellen, dass wir nur die überlappenden Daten verwenden
common_dates = df_dji.index.intersection(trade.index)
df_dji = df_dji.loc[common_dates]

# Berechnung des Portfolio-Werts basierend auf initialem Investment
initial_dji = df_dji['close'].iloc[0]
df_dji['value'] = df_dji['close'].div(initial_dji).mul(1000000)  # Skalierung auf gleiches Startkapital

## Schritt 7: Ergebnisse visualisieren

In [ ]:
# Ergebnisse zusammenführen und sicherstellen, dass die Indizes übereinstimmen
df_result = pd.DataFrame({
    'AI Agent': df_account_value.set_index('date')['account_value'],
    'DJIA': df_dji['value']
})

# Fehlende Werte entfernen
df_result = df_result.dropna()

# Plot erstellen
plt.figure(figsize=(15, 7))
df_result.plot()
plt.title('Portfolio Wert Vergleich: AI Agent vs. DJIA')
plt.xlabel('Datum')
plt.ylabel('Portfolio Wert ($)')
plt.grid(True)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Renditen berechnen und anzeigen
print("\nGesamtrenditen:")
for column in df_result.columns:
    total_return = (df_result[column].iloc[-1] - df_result[column].iloc[0]) / df_result[column].iloc[0] * 100
    print(f"{column}: {total_return:.2f}%")

## Schritt 8: Handelsaktionen analysieren

In [ ]:
# Handelsaktionen analysieren
print("\nDurchschnittliche Positionsgrößen:")
print(df_actions.mean())

# Positionsänderungen über Zeit visualisieren
plt.figure(figsize=(15, 7))
df_actions.iloc[:, :5].plot()  # Zeige erste 5 Aktien als Beispiel
plt.title('Positionsänderungen über Zeit (Erste 5 Aktien)')
plt.xlabel('Datum')
plt.ylabel('Positionsgröße')
plt.grid(True)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()